# `Pre-processing`

## 1 Setup

### 1.1 Libraries

In [194]:
import pandas as pd
import numpy as np

### 1.2 Paths

In [ ]:
DEPRESSION_PATH             = '/workspaces/depressao-ibge-pns/data/depression.csv'
DEPRESSION_DICTIONARY_PATH  = '/workspaces/depressao-ibge-pns/data/depression-dictionary.csv'

### 1.3 Dataframes

In [196]:
DEPRESSION            = pd.read_csv(DEPRESSION_PATH, header=[0, 1])
DEPRESSION_DICTIONARY = pd.read_csv(DEPRESSION_DICTIONARY_PATH)

In [197]:
DEPRESSION.head()

,V0001,V0026,V0031,C006,C00703,C008,C009,C01001,C013,C017,...,V033,V034,V03501,V03502,V03503,V036,V037,V038,V039,Y008
,Unidade da Federação,Tipo de situação censitária,Tipo de área,Sexo,Ano de nascimento,Idade do morador na data de referência,Cor ou raça,Cônjuge ou companheiro(a) mora em nesse domicílio.,Cônjuge ou companheiro(a) mora em outro domicílio.,___já viveu com cônjuge ou companheiro (a) antes?,...,Onde isso ocorreu?,"Nos últimos doze meses, você deixou de realizar quaisquer de suas atividades habituais (trabalhar, realizar afazeres domésticos, ir à escola etc.) por causa desse ato","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Hematomas, cortes, fraturas, queimaduras ou outras lesões físicas ou ferimentos?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Medo, tristeza, desânimo, dificuldade para dormir, ansiedade, depressão ou outras consequências psicológicas?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Doença sexualmente transmissível ou gravidez indesejada?","Por causa desta (s) consequência (s), você procurou algum atendimento de saúde","Por causa desta (s) consequência (s), você recebeu algum atendimento de saúde",Onde foi realizado este atendimento de saúde,"Por causa desta (s) consequência (s), você precisou ser internado por 24 horas ou mais",Qual é a sua orientação sexual?
0,11,1,1,2.0,1963.0,55.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,11,1,1,1.0,1950.0,69.0,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,1,1,1.0,1987.0,31.0,2.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,1,1,1.0,9999.0,9.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1,1,2.0,9999.0,6.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
DEPRESSION_DICTIONARY.head()

,ModuloParteAnotacao,Posicao,Tamanho,Codigo,Numero,Descricao,Dtype,Tipo,Classe,Include,Felipe,Diego
0,Parte 1 - Identificação e Controle,1.0,2.0,V0001,NaN,Unidade da Federação,category,11,Rondônia,True,True,True
1,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,12,Acre,True,True,True
2,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,13,Amazonas,True,True,True
3,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,14,Roraima,True,True,True
4,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,15,Pará,True,True,True


## 1.4 Infer Attribute V03501 Descriptions
Attribute V03501 does not have value descriptions. We'll infer them as bool ('Sim' and 'Não', respectively), due to the description and pattern of the other attributes.


In [199]:
display(DEPRESSION_DICTIONARY[DEPRESSION_DICTIONARY['Codigo'] == 'V03501'])

DEPRESSION_DICTIONARY.loc[(DEPRESSION_DICTIONARY['Codigo'] == 'V03501') & (DEPRESSION_DICTIONARY['Tipo'] == '1'), 'Classe'] = 'Sim'
DEPRESSION_DICTIONARY.loc[(DEPRESSION_DICTIONARY['Codigo'] == 'V03501') & (DEPRESSION_DICTIONARY['Tipo'] == '2'), 'Classe'] = 'Não'

DEPRESSION_DICTIONARY[DEPRESSION_DICTIONARY['Codigo'] == 'V03501']

,ModuloParteAnotacao,Posicao,Tamanho,Codigo,Numero,Descricao,Dtype,Tipo,Classe,Include,Felipe,Diego
409,Módulo V - Violência (Para pessoas de 18 ano...,1275.0,1.0,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,bool,1,NaN,True,True,NaN
410,Módulo V - Violência (Para pessoas de 18 ano...,NaN,NaN,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,NaN,2,NaN,True,True,NaN
411,Módulo V - Violência (Para pessoas de 18 ano...,NaN,NaN,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,NaN,NaN,Não aplicável,False,False,False


,ModuloParteAnotacao,Posicao,Tamanho,Codigo,Numero,Descricao,Dtype,Tipo,Classe,Include,Felipe,Diego
409,Módulo V - Violência (Para pessoas de 18 ano...,1275.0,1.0,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,bool,1,Sim,True,True,NaN
410,Módulo V - Violência (Para pessoas de 18 ano...,NaN,NaN,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,NaN,2,Não,True,True,NaN
411,Módulo V - Violência (Para pessoas de 18 ano...,NaN,NaN,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,NaN,NaN,Não aplicável,False,False,False


## 2 [Implicit `NaNs`](https://github.com/feed0/depressao-ibge-pns/issues/4)
> Useless classes such as '09 - Ignored' become NaNs

>Here we define the `depression_explicit`: pd.DataFrame(),<br>
>which will drop useless classes such as **'09 - I don't know'**

In [200]:
# Here, `9999.0` means "Not answered"
DEPRESSION.C00703.value_counts(ascending=False).head(3)

Ano de nascimento
9999.0               16342
2000.0                4434
2004.0                4225
Name: count, dtype: int64

In [201]:
# These are the Tipos we need to annull

tipos_to_remove = DEPRESSION_DICTIONARY[
    (DEPRESSION_DICTIONARY.Tipo.notna())
    & (DEPRESSION_DICTIONARY.Include == False)
]

display(
    tipos_to_remove.shape,
    tipos_to_remove.head(3)
)

(39, 12)

,ModuloParteAnotacao,Posicao,Tamanho,Codigo,Numero,Descricao,Dtype,Tipo,Classe,Include,Felipe,Diego
37,Módulo C - Características gerais dos moradores,NaN,NaN,C00703,C7,Ano de nascimento,NaN,9999,Não informado,False,False,False
46,Módulo C - Características gerais dos moradores,NaN,NaN,C009,C9,Cor ou raça,NaN,9,Ignorado,False,False,False
50,Módulo C - Características gerais dos moradores,NaN,NaN,C01001,C10a,Cônjuge ou companheiro(a) mora em nesse domicí...,NaN,9,Ignorado,False,False,False


In [202]:
# In depression['C00703']   replace 9999    with np.nan
# In depression['C009']     replace 9       with np.nan
# ...
# In depression['Q109']  replace 9       with np.nan

depression_explicit = DEPRESSION.copy()

for codigo in tipos_to_remove.Codigo:
    
    # print(f'    [CODIGO]: {codigo} ==================================================')
    # print(f'    [BEFORE]:\n{depression_explicit[codigo].value_counts(ascending=False).head(3)}\n')

    code_tipos = tipos_to_remove.Tipo[tipos_to_remove.Codigo == codigo].values.astype(int)
    # print(f'    [CODE TIPOS]: === {code_tipos} ===')

    depression_explicit[codigo] = depression_explicit[codigo].replace(code_tipos, np.nan)
    # print(f'    [AFTER]:\n{depression_explicit[codigo].value_counts(ascending=False).head(2)}\n')

In [203]:
# Check the removal of the '9999' values
depression_explicit.C00703.value_counts(ascending=False).head(3)

Ano de nascimento
2000.0               4434
2004.0               4225
2005.0               4180
Name: count, dtype: int64

## 3: [Class `mapping`](https://github.com/feed0/depressao-ibge-pns/issues/7)
depression_without_implicit C00703

### 3.1 Each Codigo has its Tipos to be cast to their corresponding Classes

In [204]:
class_mappings: pd.DataFrame = DEPRESSION_DICTIONARY.loc[DEPRESSION_DICTIONARY['Tipo'].notna(), ['Codigo', 'Tipo', 'Classe']]
class_mappings.head()

,Codigo,Tipo,Classe
0,V0001,11,Rondônia
1,V0001,12,Acre
2,V0001,13,Amazonas
3,V0001,14,Roraima
4,V0001,15,Pará


### 3.2 Remove comments from class_mappings.Tipo

In [205]:
# Tipos which contain comments
non_numeric_tipos: pd.DataFrame = class_mappings[class_mappings['Tipo'].str.contains(r'[a-zA-Z]')]
non_numeric_tipos

,Codigo,Tipo,Classe
36,C00703,ano atual - 130 a ano atual,Ano
39,C008,000 a 130,Idade (em anos)
63,C01801,000 a 130,Idade (em anos)
168,P00103,1 a 599,Quilogramas
170,P00104,1 a 599,Quilogramas
177,P02801,1 a 7,Dias
185,P035,1 a 7,Dias
206,P053,1 a 98,Anos


In [206]:
# Annull these comments in class_mappings
class_mappings['Tipo'] = class_mappings['Tipo'].replace(r'[a-zA-Z]', np.nan, regex=True)

# Check the annullation
class_mappings[class_mappings['Tipo'].isna()]

,Codigo,Tipo,Classe
36,C00703,NaN,Ano
39,C008,NaN,Idade (em anos)
63,C01801,NaN,Idade (em anos)
168,P00103,NaN,Quilogramas
170,P00104,NaN,Quilogramas
177,P02801,NaN,Dias
185,P035,NaN,Dias
206,P053,NaN,Anos


In [207]:
# Drop these new null from class_mappings
class_mappings = class_mappings.dropna(subset=['Tipo'])

# Check the drop
class_mappings[class_mappings['Tipo'].isna()]

,Codigo,Tipo,Classe


### 3 `Map` Tipos to Classes

In [208]:
# Cast to int
class_mappings['Tipo'] = class_mappings['Tipo'].astype(int)

# These are the Tipos we need to map to their corresponding Classes
display(
    "[Class Mappings]",class_mappings.Tipo.shape, class_mappings.head(3),
    "[Depression]", depression_explicit.shape, depression_explicit.head(3),
)

'[Class Mappings]'

(353,)

,Codigo,Tipo,Classe
0,V0001,11,Rondônia
1,V0001,12,Acre
2,V0001,13,Amazonas


'[Depression]'

(293726, 87)

,V0001,V0026,V0031,C006,C00703,C008,C009,C01001,C013,C017,...,V033,V034,V03501,V03502,V03503,V036,V037,V038,V039,Y008
,Unidade da Federação,Tipo de situação censitária,Tipo de área,Sexo,Ano de nascimento,Idade do morador na data de referência,Cor ou raça,Cônjuge ou companheiro(a) mora em nesse domicílio.,Cônjuge ou companheiro(a) mora em outro domicílio.,___já viveu com cônjuge ou companheiro (a) antes?,...,Onde isso ocorreu?,"Nos últimos doze meses, você deixou de realizar quaisquer de suas atividades habituais (trabalhar, realizar afazeres domésticos, ir à escola etc.) por causa desse ato","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Hematomas, cortes, fraturas, queimaduras ou outras lesões físicas ou ferimentos?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Medo, tristeza, desânimo, dificuldade para dormir, ansiedade, depressão ou outras consequências psicológicas?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Doença sexualmente transmissível ou gravidez indesejada?","Por causa desta (s) consequência (s), você procurou algum atendimento de saúde","Por causa desta (s) consequência (s), você recebeu algum atendimento de saúde",Onde foi realizado este atendimento de saúde,"Por causa desta (s) consequência (s), você precisou ser internado por 24 horas ou mais",Qual é a sua orientação sexual?
0,11,1,1,2.0,1963.0,55.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,11,1,1,1.0,1950.0,69.0,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,1,1,1.0,1987.0,31.0,2.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
# Map the Tipos to their corresponding Classes

columns = depression_explicit.columns.get_level_values(0).to_list()
class_mappings.loc[class_mappings['Codigo'] == columns[0], ['Tipo', 'Classe']]

for codigo in columns:

    # print(f'    [COLUMN]: {codigo} ==================================================')
    # print(f'    [BEFORE]:\n{depression_explicit[codigo].value_counts(ascending=False).head(3)}\n')

    # Mappings for the current column
    mappings = class_mappings.loc[class_mappings['Codigo'] == codigo, ['Tipo', 'Classe']]
    # display(mappings)

    # Mappings for the current column
    depression_explicit[codigo] = depression_explicit[codigo].replace(
        mappings['Tipo'].values.astype(int),
        mappings['Classe'].values
    )

    # print(f'    [AFTER]:\n{depression_explicit[codigo].value_counts(ascending=False).head(3)}\n')

In [210]:
depression_explicit['V0001'].value_counts(ascending=False).head(30)

Unidade da Federação
São Paulo               18504
Maranhão                17912
Minas Gerais            15421
Ceará                   15281
Rio de Janeiro          14776
Pará                    14393
Amazonas                13074
Pernambuco              12381
Paraná                  11694
Bahia                   11061
Espírito Santo          10673
Santa Catarina          10607
Alagoas                 10474
Rio Grande do Sul       10369
Paraíba                 10094
Rio Grande do Norte      9964
Piauí                    9044
Mato Grosso do Sul       8766
Goiás                    8506
Roraima                  8476
Acre                     8270
Sergipe                  8140
Distrito Federal         7609
Mato Grosso              7607
Rondônia                 7391
Amapá                    6640
Tocantins                6599
Name: count, dtype: int64

In [211]:
depression_explicit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293726 entries, 0 to 293725
Data columns (total 87 columns):
 #   Column                                                                                                                                                                                                                   Non-Null Count   Dtype  
---  ------                                                                                                                                                                                                                   --------------   -----  
 0   (V0001, Unidade da Federação)                                                                                                                                                                                            293726 non-null  object 
 1   (V0026, Tipo de situação censitária)                                                                                                           

In [212]:
depression_explicit.head(10)

,V0001,V0026,V0031,C006,C00703,C008,C009,C01001,C013,C017,...,V033,V034,V03501,V03502,V03503,V036,V037,V038,V039,Y008
,Unidade da Federação,Tipo de situação censitária,Tipo de área,Sexo,Ano de nascimento,Idade do morador na data de referência,Cor ou raça,Cônjuge ou companheiro(a) mora em nesse domicílio.,Cônjuge ou companheiro(a) mora em outro domicílio.,___já viveu com cônjuge ou companheiro (a) antes?,...,Onde isso ocorreu?,"Nos últimos doze meses, você deixou de realizar quaisquer de suas atividades habituais (trabalhar, realizar afazeres domésticos, ir à escola etc.) por causa desse ato","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Hematomas, cortes, fraturas, queimaduras ou outras lesões físicas ou ferimentos?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Medo, tristeza, desânimo, dificuldade para dormir, ansiedade, depressão ou outras consequências psicológicas?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Doença sexualmente transmissível ou gravidez indesejada?","Por causa desta (s) consequência (s), você procurou algum atendimento de saúde","Por causa desta (s) consequência (s), você recebeu algum atendimento de saúde",Onde foi realizado este atendimento de saúde,"Por causa desta (s) consequência (s), você precisou ser internado por 24 horas ou mais",Qual é a sua orientação sexual?
0,Rondônia,Urbano,Capital,Mulher,1963.0,55.0,Branca,Sim,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heterosexual
1,Rondônia,Urbano,Capital,Homem,1950.0,69.0,Parda,Sim,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rondônia,Urbano,Capital,Homem,1987.0,31.0,Preta,Não,Não,Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Rondônia,Urbano,Capital,Homem,NaN,9.0,Preta,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rondônia,Urbano,Capital,Mulher,NaN,6.0,Parda,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Rondônia,Urbano,Capital,Mulher,NaN,4.0,Preta,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Rondônia,Urbano,Capital,Mulher,1985.0,33.0,Branca,Não,Não,Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Rondônia,Urbano,Capital,Homem,2002.0,17.0,Branca,Não,Não,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Rondônia,Urbano,Capital,Mulher,2004.0,15.0,Parda,Não,Não,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4 Dropna all NaN
Drop rows where all columns are NaN.

In [213]:
total = depression_explicit.shape[0]
depression_explicit = depression_explicit.dropna(how="all")
loss = (total - depression_explicit.shape[0])/total
print(f"Data removed with all ({depression_explicit.shape[1]} columns) NaN: {loss}%")



Data removed with all (87 columns) NaN: 0.0%


## `TODO`: D`types`
Dtypes are already included at DEPRESSION_DICTIONARY['Dtype'] for each Codigo attribute

In [214]:
# # SELECT * FROM DEPRESSION_DICTIONARY
# # WHERE Codigo == 'V03501'
# DEPRESSION_DICTIONARY[DEPRESSION_DICTIONARY.Codigo == 'V03501']

## `TODO` Drop columns without class descriptions
Codigo `V03501` is not correctly described in the dictionary-pns-2019-cleaned.csv file.

In [215]:
# # V03501
# PNS_DICTIONARY_PATH[PNS_DICTIONARY_PATH.Codigo == 'V03501']

In [190]:
# # Original shape
# print(PNS_DICTIONARY_PATH.shape)

# # Drop Codigo 'V03501'
# PNS_DICTIONARY_PATH = PNS_DICTIONARY_PATH[PNS_DICTIONARY_PATH.Codigo != 'V03501']

# # Shape after dropping
# print(PNS_DICTIONARY_PATH.shape)